In [80]:
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
from io import StringIO

In [106]:
# Unncoment interested subject 
subject = 'publications'
subject = 'Horizon'
subject = 'EIC'

In [107]:
if subject == 'publications':
    path_original = '/export/data_ml4ds/AI4U/Datasets/ResearchPortal/20231005/parquet_preprocessed/publications.parquet'
    path_lem = '/export/data_ml4ds/AI4U/Datasets/ResearchPortal/lemmatized/publications_lemmatized.parquet'

    topic_model_folder = 'publications_calls_topicmodels'
    topic_model = 'publications_calls_44tpc'
    inference_folder = 'publications_inference'
    id_col = 'actID'
    n_topics = 44
    save_path = '/export/usuarios_ml4ds/mbalairon/github/recommendation_system/publications_topics.parquet'
    
elif subject == 'Horizon':
    topic_model_folder = 'publications_calls_topicmodels'
    topic_model = 'publications_calls_44tpc'
    n_topics = 44
    inference_folder = 'horizon_inference' 
    id_col = 'Call'
    path_original = '/export/data_ml4ds/AI4U/Datasets/work_programmes/horizon_work_programmes.parquet'
    path_lem = '/export/data_ml4ds/AI4U/Datasets/work_programmes/work_programmes_lematized/horizon_work_programmes_lematized'
    save_path = '/export/usuarios_ml4ds/mbalairon/github/recommendation_system/horizon_topics.parquet'

elif subject == 'EIC':
    topic_model_folder = 'publications_calls_topicmodels'
    topic_model = 'publications_calls_44tpc'
    n_topics = 44
    inference_folder = 'eic_inference' 
    id_col = 'id'
    path_original = '/export/data_ml4ds/AI4U/Datasets/work_programmes/EIC_work_programmes.parquet'
    path_lem = '/export/data_ml4ds/AI4U/Datasets/work_programmes/work_programmes_lematized/EIC_work_programmes_lematized'
    save_path = '/export/usuarios_ml4ds/mbalairon/github/recommendation_system/eic_topics.parquet'


In [108]:
# define header of the document topic decomposition
head = ['doc', id_col]

for i in range (n_topics):
    head.append('tpc_' + str(i))
    

In [109]:
# Load necessary files
df_lem = pd.read_parquet(path_lem)
inferences = pd.read_csv('/export/usuarios_ml4ds/mbalairon/{}/TMmodels/{}/TMinference/{}/doc-topics-inf.txt'.format(topic_model_folder, topic_model, inference_folder), sep = '\t', names=head).drop(0, axis=0)
tpc_keys = pd.read_csv('/export/usuarios_ml4ds/mbalairon/{}/TMmodels/{}/TMmodel/tpc_labels.txt'.format(topic_model_folder, topic_model), names=['label'])
df_original = pd.read_parquet(path_original)


In [110]:
# Add a column with a vector containig the complete topic decomposition 
inferences['tpc_decomposition'] = inferences[['tpc_{}'.format(i) for i in range(n_topics)]].apply(tuple, axis=1)
inferences['tpc_decomposition_dict'] = inferences['tpc_decomposition'].apply(lambda tupla: {i: valor for i, valor in enumerate(tupla)})
inferences['tpc_decomposition_dict'] = inferences['tpc_decomposition_dict'].apply(lambda d: dict(sorted(d.items(), key=lambda item: item[1], reverse=True)))
inferences['tpc_ordered'] = inferences['tpc_decomposition_dict'].apply(lambda d: ', '.join(map(str, d.keys())))
inferences['tpc_ordered'] = inferences['tpc_ordered'].apply(lambda indices: ', '.join(tpc_keys['label'][int(idx)] for idx in indices.split(', ')))


In [111]:
inferences[[id_col, 'tpc_decomposition', 'tpc_decomposition_dict', 'tpc_ordered']].shape

(8, 4)

In [112]:
df_lem.shape

(8, 7)

In [113]:
df = pd.merge(df_lem, inferences[[id_col, 'tpc_decomposition', 'tpc_decomposition_dict', 'tpc_ordered']], on=id_col, how='outer')

In [101]:
if subject == 'publications':
    df = pd.merge(df, df_original[[id_col, 'Title']], on='actID', how='inner')

In [114]:
df.shape

(8, 10)

In [115]:
df['tpc_decomposition_dict'] = df['tpc_decomposition_dict'].apply(lambda d: {str(k): v for k, v in d.items()} if isinstance(d, dict) else d)
df.to_parquet(save_path, index=False)